In [ ]:
import pandas as pd
import transformers
import re
import torch
from datasets import Dataset
from unsloth import FastLanguageModel
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
from huggingface_hub import interpreter_login
from sklearn.utils import resample
import os
from dotenv import load_dotenv

#### Acknowledgment
The code is adapted from unlslothai's work available at https://github.com/unslothai/unsloth

## Data sampling

### Generate confidence scores

In [ ]:
# load all remaining data (data not included in prompt selection or prompting experiment): text, label
data = pd.read_csv("")

In [ ]:
# load model
# Authenticate with Huggingface token
#!git config --global credential.helper store
interpreter_login()

model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct" 

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto"
)

In [ ]:
# prompt for confidence scores
sentiment_v3_prompt_confidence = """
Du er en gennemsnitlig dansk nyhedsforbruger. Du får en overskrift og underoverskrift på en artikel, og skal tildele den en kategori svarende til det sentiment den fremkalder. 
Her er nogle generelle principper, du skal følge:
Du skal bruge din viden om det danske samfund og annotere artiklen som du forestiller dig, at den gennemsnitlige dansker ville gøre det.
Undgå at lade dig påvirke af personlige holdninger og bias. Nogle artikler udtrykker holdninger som du måske er uenig med, men det må ikke påvirke vurderingen af artiklens sentiment.
Du skal tildele artiklen dens mest dominerende sentiment, hvis den vurderes at indeholde flere sentiments.
Der findes både explicit og implicit sentiment. Explicit sentiment afspejler ofte nogens indre tilstand (f.eks. tro, holdninger, tanker, følelser osv.). Implicit sentiment afspejler derimod ofte en kendsgerning, der fører til et positivt eller negativt sentiment omkring et emne (f.eks. en person eller begivenhed). Artikler fremkalder ofte implicit sentiment ved at fremhæve gode eller dårlige begivenheder. F.eks. er begivenheder som fødsler, at blive gift eller forfremmet gode begivenheder, mens f.eks. død og sygdom er dårlige begivenheder.
Du skal bruge din generelle viden om følelsesladede udtryk på dansk. Vær opmærksom på, at sentiment kan forekomme ved ord, der normalt ikke betragtes som følelsesladede, f.eks. familie, løn og ansættelse. Nogle begivenheder er også forbundet med sentiment, f.eks. 1. verdenskrig eller Covid-19.
Kategorier: ”Positiv”: Fremkalder en overordnet positiv sentiment. Stemninger som optimisme, tilfredshed og selvsikkerhed betragtes som positive. ”Negativ”: Fremkalder en overordnet negativ sentiment. Stemninger som vrede, skuffelse og tristhed betragtes som negative. ”Neutral”: Fremkalder hverken en positiv eller negativ sentiment. Enten ingen sentiment eller tvetydig sentiment. 
Giv også en confidence score med to decimaler fra 0.00 til 1.00, der repræsenterer hvor sikker du er i din vurdering af sentiment, hvor 0 er meget usikker og 1 er meget sikker.
Giv et præcist svar i json: {{sentiment: ”kategori”, "confidence": "score"}}.
"""

In [ ]:
# Function to run Llama with few-shot prompts
def fewshot_sentiment_annotation(text, prompt, fewshot_dataset):
    # Sample few-shot examples from dataset without current article
    fewshot_dataset = fewshot_dataset[fewshot_dataset["text"] != text]
    fewshot_examples = fewshot_dataset.sample(3).to_dict(orient="records")
    # Create few-shot part of prompt
    fewshot_examples = "\n".join([f"Artikel: {example["text"]}. Artiklen fremkalder dette sentiment: {example["label"]}." for example in fewshot_examples])
    
    messages = [
    {"role": "system", "content": f"{prompt}. Her er tre eksempler på artikler og deres sentiment: {fewshot_examples}"},
    {"role": "user", "content": f"Artikel: {text}. Artiklen fremkalder dette sentiment: "},
    ]

    outputs = pipeline(
        messages,
        max_new_tokens=256,
    )
    
    # Return the generated content
    return outputs[0]["generated_text"][-1]["content"]

In [ ]:
# Apply few-shot
data["llm_annotation"] = data["text"].apply(lambda text: fewshot_sentiment_annotation(text, prompt=sentiment_v3_prompt_confidence, fewshot_dataset=data))

In [ ]:
# extract confidence scores
def extract_confidence(value):
    match = re.search(r'"confidence":\s*"?(\d+\.\d+)"?', str(value))
    if match:
        return int(float(match.group(1)))
    return None  # Return None if no match is found

data["confidence_scores"] = data["llm_annotation"].apply(extract_confidence)

In [ ]:
# map labels to integers
def map_sentiment_to_int(annotation):
    if "Negativ" in annotation or "negativ" in annotation:
        return 0
    elif "Neutral" in annotation or "neutral" in annotation:
        return 1
    elif "Positiv" in annotation or "positiv" in annotation:
        return 2
    else:
        return -1 

data["label"] = data["label"].map(map_sentiment_to_int)
data["llm_annotation"] = data["llm_annotation"].map(map_sentiment_to_int)

### Random sampling

In [ ]:
data = data.sample(n=300)

### Selective sampling

In [ ]:
# Define bins and labels for confidence intervals
bins = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
labels = ["0-0.1", "0.1-0.2", "0.2-0.3", "0.3-0.4", "0.4-0.5", "0.5-0.6", "0.6-0.7", "0.7-0.8", "0.8-0.9", "0.9-1.0"]
data["confidence_interval"] = pd.cut(data["confidence_scores"], bins=bins, labels=labels, include_lowest=True)

# Determine if LLM annotation is correct
data["is_correct"] = data["llm_annotation"] == data["label"]

# Calculate the distribution of examples across intervals and correctness in the full dataset
full_distribution = data.groupby(["confidence_interval", "is_correct"]).size() / len(data)

# Calculate target sample sizes based on this distribution
total_samples = 300
sample_sizes = (full_distribution * total_samples).round().astype(int)

# Initialize an empty DataFrame for the sampled data
sampled_df = pd.DataFrame()

# Sample 300 examples based on the calculated distribution
for (interval, correct), size in sample_sizes.items():
    # Only sample if the size is greater than zero
    if size > 0:
        # Get the subset of data for this interval and correctness
        group_data = data[(data["confidence_interval"] == interval) & (data["is_correct"] == correct)]
        
        # Set `replace=True` only if we need more samples than available in this group
        replace = size > len(group_data)
        
        # Sample the data
        sampled_group = resample(
            group_data,
            n_samples=size,
            random_state=42,
            replace=replace
        )
        sampled_df = pd.concat([sampled_df, sampled_group])

# Reset index for the sampled DataFrame
sampled_df.reset_index(drop=True, inplace=True)

data = sampled_df

## Fine-tune Llama

In [ ]:
max_seq_length = 5000 
dtype = None 
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    token = os.getenv("HUGGINGFACE_TOKEN"),
)

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = True, 
    loftq_config = None,
)

In [ ]:
# format fine-tuning data to chat template
data["text"] = data["messages"].apply(lambda chat: tokenizer.apply_chat_template(chat, tokenize=False))

# convert to Huggingface dataset
data_dict = {"text": data["text"].tolist()}
dataset = Dataset.from_dict(data_dict)

In [ ]:
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 8,
        warmup_steps = 10,
        max_steps = 100,
        learning_rate = 1e-5, 
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 5,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",
    ),
)

In [ ]:
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

In [ ]:
trainer_stats = trainer.train()

In [ ]:
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

## Inference with fine-tuned model

In [ ]:
# Load evaluation data
evaluation_data = pd.read_csv("")

In [ ]:
# Load model
model = FastLanguageModel.for_inference(model)

In [ ]:
# V0 zeroshot 
def run_v0_sentiment_inference(text, model):
    # Define the chat input, inserting the text
    chat_input = [
        f'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n'
        f'Cutting Knowledge Date: December 2023\nToday Date: 26 Jul 2024\n\n'
        f'Du er en gennemsnitlig dansk nyhedsforbruger. Du får en overskrift og underoverskrift'
        f'på en artikel, og skal tildele den en kategori svarende til det sentiment den fremkalder.'
        f'Kategorier: ”Positiv”, ”Negativ”, ”Neutral”'
        f'Giv et præcist svar i json: {{sentiment: ”kategori”}}.<|eot_id|><|start_header_id|>user<|end_header_id>\n\n'
        f'Artikel: {text} \nArtiklen fremkalder dette sentiment:<|eot_id|><|start_header_id|>assistant<|end_header_id>\n\n'
    ]

    # Tokenize the chat input for the model
    inputs = tokenizer(chat_input, return_tensors="pt").to("cuda")

    # Generate the output
    outputs = model.generate(**inputs, max_new_tokens=64, use_cache=True)

    # Decode the generated output
    decoded_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return decoded_outputs

evaluation_data["v0-zeroshot-annotation"] = evaluation_data["text"].apply(lambda text: run_v0_sentiment_inference(text, model=model))

In [ ]:
# V3 few-shot
def run_v3_fewshot_sentiment_inference(text, model, fewshot_dataset):
    # Sample few-shot examples from dataset without current article
    fewshot_dataset = fewshot_dataset[fewshot_dataset["text"] != text]
    fewshot_examples = fewshot_dataset.sample(1).to_dict(orient="records")
    # Create few-shot part of prompt
    fewshot_example = "\n".join([f"Artikel: {example['text']}. Artiklen fremkalder dette sentiment: {example['label']}." for example in fewshot_examples])
    
    # Define the chat input, inserting the text. v3 prompt    
    chat_input = [
        f'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n'
        f'Cutting Knowledge Date: December 2023\nToday Date: 26 Jul 2024\n\n'
        f'Du er en gennemsnitlig dansk nyhedsforbruger. Du får en overskrift og underoverskrift på en artikel, og skal tildele den en kategori svarende til det sentiment den fremkalder. Her er nogle generelle principper, du skal følge: Du skal bruge din viden om det danske samfund og annotere artiklen som du forestiller dig, at den gennemsnitlige dansker ville gøre det. Undgå at lade dig påvirke af personlige holdninger og bias. Nogle artikler udtrykker holdninger som du måske er uenig med, men det må ikke påvirke vurderingen af artiklens sentiment. Du skal tildele artiklen dens mest dominerende sentiment, hvis den vurderes at indeholde flere sentiments. Der findes både explicit og implicit sentiment. Explicit sentiment afspejler ofte nogens indre tilstand (f.eks. tro, holdninger, tanker, følelser osv.). Implicit sentiment afspejler derimod ofte en kendsgerning, der fører til et positivt eller negativt sentiment omkring et emne (f.eks. en person eller begivenhed). Artikler fremkalder ofte implicit sentiment ved at fremhæve gode eller dårlige begivenheder. F.eks. er begivenheder som fødsler, at blive gift eller forfremmet gode begivenheder, mens f.eks. død og sygdom er dårlige begivenheder. Du skal bruge din generelle viden om følelsesladede udtryk på dansk. Vær opmærksom på, at sentiment kan forekomme ved ord, der normalt ikke betragtes som følelsesladede, f.eks. familie, løn og ansættelse. Nogle begivenheder er også forbundet med sentiment, f.eks. 1. verdenskrig eller Covid-19. '
        f'Kategorier: ”Positiv”: Fremkalder en overordnet positiv sentiment. Stemninger som optimisme, tilfredshed og selvsikkerhed betragtes som positive. ”Negativ”: Fremkalder en overordnet negativ sentiment. Stemninger som vrede, skuffelse og tristhed betragtes som negative. ”Neutral”: Fremkalder hverken en positiv eller negativ sentiment. Enten ingen sentiment eller tvetydig sentiment. '
        f'Her er eksempler på en artikler og sentiment: {fewshot_example}'
        f'Giv et præcist svar i json: {{sentiment: ”kategori”}}.<|eot_id|><|start_header_id|>user<|end_header_id>\n\n'
        f'Artikel: {text} \nArtiklen fremkalder dette sentiment:<|eot_id|><|start_header_id|>assistant<|end_header_id>\n\n'
    ]

    # Tokenize the chat input for the model
    inputs = tokenizer(chat_input, return_tensors="pt").to("cuda")

    # Generate the output
    outputs = model.generate(**inputs, max_new_tokens=64, use_cache=True)

    # Decode the generated output
    decoded_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return decoded_outputs

evaluation_data["v3-fewshot-annotation"] = evaluation_data["text"].apply(lambda text: run_v3_fewshot_sentiment_inference(text, model=model, fewshot_dataset=evaluation_data))

: 

In [ ]:
# additionally, extract [-30:]
evaluation_data["v3-fewshot-llm_annotation"] = evaluation_data["v3-fewshot-annotation"].apply(lambda text: text[0][-30:])
evaluation_data["v0-fewshot-llm_annotation"] = evaluation_data["v0-zeroshot-annotation"].apply(lambda text: text[0][-30:])

In [ ]:
# save evaluation results in csv
evaluation_data.to_csv("")